<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/XGBoost_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

In [ ]:
raw_df = pd.read_csv("../A2W/데이터/data_submission/merge_final_real.csv", encoding='utf-8-sig')

In [ ]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

# 1일 후

In [ ]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)

In [ ]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.2, max_depth = 6)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

In [ ]:
xgb_pred = xgb.predict(X_test)
metrics(y_test, xgb_pred)

# 2일 후

In [ ]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','2day_yn']]
cleaned_df_2 = select_df.fillna(0)

In [ ]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)

In [ ]:
# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# 최고 하이퍼 파라미터 값으로 예측 실시
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.2, max_depth = 6)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

In [ ]:
xgb_pred = xgb.predict(X_test)
metrics(y_test, xgb_pred)